# 问题2
## 数据预处理与清洗
1. 提取数据+计算利润+计算加成率
2. 3sigma异常值处理


## 各蔬菜品类的销售总量与成本加成定价的关系
1. 对于每个类进行建立单独的ADF检验
2. 寻找最优滞后阶数
3. 对于每个类进行建立单独的VAR向量自回归模型,得出各蔬菜品类的销售总量与成本加成定价的关系
4. VAR模型稳定性检验



## 各蔬菜品类未来一周(2023 年7 月 1-7 日)的日补货总量和定价策略
> 前提:忽略库存,理想情况下日补货总量将等于日销售量为最佳
## 因果检验
1. ADF检验确认各时间序列均是平稳序列(避免伪回归)
2. 使用之前的每类的最优滞后阶数(通过对FPE、AIC、SC、HQ的指标进行评价)
3. 格兰杰因果检验变量之间是否存在因果关系(证明销量(千克)可以引起销售单价(元/千克)变化)


### 售价关系与(销售量+批发)的关系
1. 建立回归模型
2. F检验
3. $R^2$检验
4. 得到公式:$$售价=a*销售量+b*批发价+c$$
            $$s=a*x+b*p+c\space\space\space\space\space\space\space\space(1)$$


In [18]:
import pandas as pd
from pulp import LpVariable, LpProblem, LpMinimize, LpStatus, value
from sklearn.metrics import r2_score


In [42]:

# 从Excel文件读取数据
data = pd.read_excel('Q2.xlsx')
groups=data.groupby("分类名称")
data

,销售日期,分类名称,销量(千克),批发价格(元/千克),销售单价(元/千克),利润(元/千克),小分类编码,小分类名称,平均损耗率(%)_小分类编码_不同值,加成率
0,2020-07-01,水生根茎类,4.850,5.65,10.0,4.35,1011010402,水生根茎类,13.65,0.769912
1,2020-07-01,花叶类,205.402,2.10,3.2,1.10,1011010101,花叶类,12.83,0.523810
2,2020-07-01,花菜类,46.640,9.23,14.0,4.77,1011010201,花菜类,15.51,0.516793
3,2020-07-01,茄类,35.374,1.63,4.0,2.37,1011010501,茄类,6.68,1.453988
4,2020-07-01,辣椒类,76.715,4.32,7.6,3.28,1011010504,辣椒类,9.24,0.759259
...,...,...,...,...,...,...,...,...,...,...
6469,2023-06-30,花叶类,130.464,2.60,4.8,2.20,1011010101,花叶类,12.83,0.846154
6470,2023-06-30,花菜类,28.087,7.59,11.2,3.61,1011010201,花菜类,15.51,0.475626
6471,2023-06-30,茄类,24.530,3.20,5.2,2.00,1011010501,茄类,6.68,0.625000
6472,2023-06-30,辣椒类,82.286,2.11,5.8,3.69,1011010504,辣椒类,9.24,1.748815


In [57]:
pdtime=pd.read_excel('time.xlsx')
pdtime

,天数,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
0,1,11.869472,3.317718,7.785520,4.145110,4.926485,6.719549
1,2,11.926415,3.316590,7.810809,4.146312,5.242987,7.114341
2,3,12.385582,3.315461,7.814275,4.147514,5.278217,7.142660
3,4,12.423005,3.314333,7.814943,4.148716,5.281800,7.143429
4,5,12.461304,3.313204,7.815252,4.149918,5.281823,7.142126
5,6,12.470742,3.312076,7.815516,4.151120,5.281446,7.140667
6,7,12.478412,3.310947,7.815773,4.152322,5.281023,7.139196


#### R方检验

In [58]:
def getRsquared(A,B,C,price,wholesale_price,quantity):
    # 获取线性规划模型的结果
    A_value = value(A)
    B_value = value(B)
    C_value = value(C)

    # 使用线性规划模型的结果进行预测
    predicted_price = [A_value * q + B_value * w + C_value for q, w in zip(quantity, wholesale_price)]

    # 计算R方
    r_squared = r2_score(price, predicted_price)

    # 打印R方值
    print("R-squared:", r_squared)

### 利润公式
$$利润=销量*(售价-批发)$$
$$w=x*(s-p)\space\space\space\space\space\space\space\space(2)$$
### 批发价预测(时间序列ARIMA预测)
1. ARIMA模型要求序列满足平稳性，查看ADF检验结果，根据分析t值，分析其是否可以显著性地拒绝序列不平稳的假设(P<0.05)。
2. 查看差分前后数据对比图，判断是否平稳（上下波动幅度不大），同时对时间序列进行偏（自相关分析），根据截尾情况估算其p、q值。
3. ARIMA模型要求模型具备纯随机性，即模型残差为白噪声，查看模型检验表，根据Q统计量的P值(P>0.05)对模型白噪声进行检验，也可以结合信息准则AIC和BIC值进行分析（越低越好），也可以通过模型残差ACF/PACF图进行分析根据模型参数表，得出模型公式结合时间序列分析图进行综合分析，得到向后预测的阶数结果。
### 联立
联立公式(1)(2),即可得到一个一元二次方程:
$$w=ax^2+(b-1)px+cx\space\space\space\space\space\space\space\space(满足a<0且b>1)$$ 
显然当且仅当$x=\frac{1-b}{2a}p$有w最大值

In [66]:
def predict(A, B, C, name):
    # 创建空的数据框
    result_df = pd.DataFrame(columns=['日期', '销量', '利润', '售价'])

    # 预测与结果
    for i, p in enumerate(pdtime[name].tolist()):
        print("第", i, "天:")

        # 销量
        x = (1 - B) / (2 * A) * p
        # 利润
        w = A * (x ** 2) + (B - 1) * p * x + C * x
        # 售价
        s = A * x + B * p + C

        # 将预测结果添加到数据框中
        result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)

        print("     销量:", x)
        print("     利润:", w)
        print("     售价:", s)

    # 将数据框保存为Excel文件
    result_df.to_excel('预测结果'+name+'.xlsx', index=False)

#### 带约束优化的线性回归方程

In [63]:
def getLp(quantity,wholesale_price,price,name=None):

    # 创建线性规划问题
    problem = LpProblem("Constrained Linear Regression", LpMinimize)

    # 创建变量
    A = LpVariable('A')
    B = LpVariable('B')
    C = LpVariable('C')
    E = [LpVariable(f'E{i}') for i in range(len(price))]

    # 创建约束条件
    constraint1 = A <= -0.0001
    constraint2 = B >= 1.000001

    # 创建目标函数
    error = sum(E[i] for i in range(len(price)))

    # 添加约束条件和目标函数到问题中
    problem += error
    problem += constraint1
    problem += constraint2

    # 添加误差约束条件
    for q, w, p, e in zip(quantity, wholesale_price, price, E):
        problem += e >= p - (A * q + B * w + C)
        problem += e >= (A * q + B * w + C) - p

    # 求解问题
    problem.solve()

    # 打印结果
    print(name)
    print("Status:", LpStatus[problem.status])
    print("A =", value(A))
    print("B =", value(B))
    print("C =", value(C))
    getRsquared(A,B,C,price,wholesale_price,quantity)
    predict(value(A),value(B),value(C),name)
    

In [67]:
for name,group in groups:
    quantity =group['销量(千克)'].tolist()
    wholesale_price = group['批发价格(元/千克)'].tolist()
    price = group['销售单价(元/千克)'].tolist()
    getLp(quantity,wholesale_price,price,name)

d:\python3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


水生根茎类
Status: Optimal
A = -0.011386757
B = 1.4025728
C = 0.87115986
R-squared: 0.9026923445454548
第 0 天:
     销量: 209.81946080159994
     利润: 684.0792294296486
     售价: 15.129794702598046
第 1 天:
     销量: 210.82606974250638
     利润: 689.7775906018759
     售价: 15.198200508536374
第 2 天:
     销量: 218.94287460303602
     利润: 736.569626442682
     售价: 15.749791653384241
第 3 天:
     销量: 219.60440895750364
     利润: 740.4493873104811
     售价: 15.794747334616817
第 4 天:
     销量: 220.28142497030126
     利润: 744.4302637792844
     售价: 15.840755098015855
第 5 天:
     销量: 220.44825824766522
     利润: 745.4128520008372
     售价: 15.85209253436441
第 6 天:
     销量: 220.58385139508746
     利润: 746.2119139876341
     售价: 15.861306995067274


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will

花叶类
Status: Optimal
A = -0.0001
B = 1.4612691
C = 0.6727909
R-squared: 0.9247966356753622
第 0 天:
     销量: 7651.803837156778
     利润: 11003.07418645688
     售价: 4.755689221967677
第 1 天:
     销量: 7649.201330485699
     利润: 10997.34114684909
     售价: 4.75430055985004
第 2 天:
     销量: 7646.598823814621
     利润: 10991.609461849494
     售价: 4.752911897732401
第 3 天:
     销量: 7643.996317143542
     利润: 10985.879131458092
     售价: 4.751523235614765
第 4 天:
     销量: 7641.393810472465
     利润: 10980.150155674888
     售价: 4.750134573497126
第 5 天:
     销量: 7638.791303801386
     利润: 10974.422534499878
     售价: 4.748745911379488
第 6 天:
     销量: 7636.188797130308
     利润: 10968.696267933059
     售价: 4.74735724926185


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will

...

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will

辣椒类
Status: Optimal
A = -0.0001
B = 1.4061795
C = 0.57701007
R-squared: 0.9227535317797613
第 0 天:
     销量: 10005.186995850796
     利润: 15783.470331033142
     售价: 6.504014225326099
第 1 天:
     销量: 10647.969802245583
     利润: 17481.911891904994
     售价: 6.884794351280169
第 2 天:
     销量: 10719.51708166146
     利润: 17676.073948058856
     售价: 6.927178468895965
第 3 天:
     销量: 10726.794811968173
     利润: 17695.881319136122
     售价: 6.931489746415028
第 4 天:
     销量: 10726.841593309708
     利润: 17696.00867543046
     售价: 6.931517459360901
第 5 天:
     销量: 10726.074822608894
     利润: 17693.921294239237
     售价: 6.93106322965667
第 6 天:
     销量: 10725.216519395713
     利润: 17691.584873413543
     售价: 6.930554776719891


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will

食用菌
Status: Optimal
A = -0.0020576562
B = 1.4962597
C = 0.63704962
R-squared: 0.9511842312744401
第 0 天:
     销量: 810.3009271333862
     利润: 1867.2334282938823
     售价: 9.02391951239861
第 1 天:
     销量: 857.9083641213067
     利润: 2060.9790728436815
     售价: 9.516671484608224
第 2 天:
     销量: 861.3232978030959
     利润: 2075.235182397373
     售价: 9.552017124137056
第 3 天:
     销量: 861.4159376459676
     利润: 2075.622588758656
     售价: 9.55297597569825
第 4 天:
     销量: 861.258814847522
     利润: 2074.9655448949015
     售价: 9.551349705230617
第 5 天:
     销量: 861.0829154507326
     利润: 2074.2301030508866
     售价: 9.549529091054296
第 6 天:
     销量: 860.905604471155
     利润: 2073.488888194397
     售价: 9.547693866551377


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'日期': i, '销量': x, '利润': w, '售价': s}, ignore_index=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25556\1865991907.py:17: FutureWarning: The frame.append method is deprecated and will